In [1]:
import pandas as pd
import numpy as np
import re
import os
from ast import literal_eval
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [27]:
pickle_location = r'/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/d:\temp\pickle_file'
df = pd.read_pickle(pickle_location)


In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/cards_data.csv')
df.sample(5)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
21139,569dab90-a0d8-4c0a-8e32-b2cb3aec39a6,Splitting the Powerstone,3.0,{2}{U},['U'],False,['Sorcery'],['none'],"As an additional cost to cast this spell, sacr...",['no keywords'],non-creature,non-creature,False,uncommon,False,2022-11-18,True,0.01
19131,fc32c801-cc84-4d3a-a14f-67e28ccb95f2,Screaming Seahawk,5.0,{4}{U},['U'],False,['Creature'],['Bird'],Flying\nWhen Screaming Seahawk enters the batt...,['Flying'],2,2,False,common,False,2014-06-06,True,0.03
13135,0a03ba5a-ac27-4fce-9eaf-b029ab26f9e1,Lumengrid Warden,2.0,{1}{U},['U'],False,['Creature'],"['Human', 'Wizard']",NaN,['no keywords'],1,3,False,common,False,2007-07-13,True,0.03
25439,5df7b710-d44c-4ebc-be5b-f81d697086c4,Windbrisk Raptor,7.0,{5}{W}{W},['W'],False,['Creature'],['Bird'],Flying\nAttacking creatures you control have l...,['Flying'],5,7,False,rare,False,2008-05-02,True,2.25
6782,dd7904ba-0764-4dd3-9c78-35f799e99049,Endure,5.0,{3}{W}{W},['W'],False,['Instant'],['none'],Prevent all damage that would be dealt to you ...,['no keywords'],non-creature,non-creature,False,uncommon,False,2008-07-25,True,0.18


In [3]:
df.head()

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
0,e882c9f9-bf30-46b6-bedc-379d2c80e5cb,+2 Mace,2.0,{1}{W},['W'],False,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],non-creature,non-creature,False,common,False,2021-07-23,True,0.02
1,2a83882c-3e03-4e85-aaac-97fa1d08a772,Aarakocra Sneak,4.0,{3}{U},['U'],False,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,False,common,False,2022-06-10,True,0.06
2,c9f1fc97-00c0-492b-a4a3-b179afc2f95d,Abaddon the Despoiler,5.0,{2}{U}{B}{R},"['B', 'R', 'U']",True,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,False,mythic,False,2022-10-07,False,2.81
3,4945031e-1158-474c-9e50-1ec817acc767,Abandoned Outpost,0.0,{0},['No Colors'],False,['Land'],['none'],Abandoned Outpost enters the battlefield tappe...,['no keywords'],non-creature,non-creature,False,common,False,2001-10-01,True,0.14
4,23b587f3-5eac-45a7-a048-dd73d2a2b74f,Abandoned Sarcophagus,3.0,{3},['C'],False,['Artifact'],['none'],You may cast spells that have a cycling abilit...,['no keywords'],non-creature,non-creature,False,rare,False,2020-04-17,False,0.09


In [4]:
df['types_embedding'][0].shape

KeyError: 'types_embedding'

In [5]:
df['sub_type_embedding'][0].shape

KeyError: 'sub_type_embedding'

In [6]:
df['keywords_embedding'][0].shape

KeyError: 'keywords_embedding'

In [7]:
df['text_embeddings']= df['text_embeddings'].apply(np.array)

KeyError: 'text_embeddings'

In [8]:
df['text_embeddings'][0].shape

KeyError: 'text_embeddings'

In [9]:
def reshape_array(arr):
    return arr.reshape(768,)

df['text_embeddings'] = df['text_embeddings'].apply(reshape_array)
df['text_embeddings'][0].shape

KeyError: 'text_embeddings'

In [10]:
embedding_columns = ['types_embedding', 'sub_type_embedding', 'keywords_embedding', 'text_embeddings']
df['combined_embedding'] = df[embedding_columns].apply(np.concatenate, axis=1)

KeyError: "None of [Index(['types_embedding', 'sub_type_embedding', 'keywords_embedding',\n       'text_embeddings'],\n      dtype='object')] are in the [columns]"

In [11]:
df.combined_embedding[0].shape

AttributeError: 'DataFrame' object has no attribute 'combined_embedding'

In [12]:
df.columns

Index(['id', 'name', 'cmc', 'mana_cost', 'colors', 'legendary', 'types',
       'sub_types', 'oracle_text', 'keywords', 'power', 'toughness',
       'textless', 'rarity', 'reserved', 'released_at', 'booster', 'usd'],
      dtype='object')

In [13]:
df = df[['name','B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
         'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon',
       'booster', 'cmc' ,'power', 'toughness',
       'combined_embedding', 'released_at', 'usd']]

KeyError: "['B', 'C', 'G', 'N', 'R', 'U', 'W', 'rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special', 'rarity_uncommon', 'combined_embedding'] not in index"

In [14]:
df.sample(1)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
7405,3185eeac-39ae-4e6e-bfdd-4b8edd984af1,Farbog Boneflinger,5.0,{4}{B},['B'],False,['Creature'],['Zombie'],When Farbog Boneflinger enters the battlefield...,['no keywords'],2,2,False,uncommon,False,2016-08-26,True,0.22


In [15]:
df['released_at'] = pd.to_datetime(df['released_at'])
reference_date = df['released_at'].min()
df['released_at'] = (df['released_at'] - reference_date).dt.days
df['released_at']

0        10096
1        10418
2        10537
3         2861
4         9634
         ...  
26047     7604
26048     4258
26049    10446
26050    10376
26051    10908
Name: released_at, Length: 26052, dtype: int64

In [41]:
df.columns

Index(['name', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'reserved',
       'rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special',
       'rarity_uncommon', 'booster', 'cmc', 'power', 'toughness',
       'combined_embedding', 'usd', 'days_since_release'],
      dtype='object')

In [42]:
df = df[['name', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'reserved',
       'rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special',
       'rarity_uncommon', 'booster', 'cmc', 'power', 'toughness','days_since_release',
       'combined_embedding', 'usd']]
df.sample(1)

,name,B,C,G,N,R,U,W,legendary,reserved,...,rarity_rare,rarity_special,rarity_uncommon,booster,cmc,power,toughness,days_since_release,combined_embedding,usd
3153,Carom,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,1,2.0,0,0,4538,"[-0.009292987175285816, 0.00402201758697629, 0...",0.12


#### Fill X power and toughness with -1

In [43]:
df['power'] = pd.to_numeric(df['power'], errors='coerce')
df['toughness'] = pd.to_numeric(df['toughness'], errors='coerce')
index = df[df['power'].isna() | df['toughness'].isna()][['power', 'toughness']].index
df[df['power'].isna() | df['toughness'].isna()][['power', 'toughness']]

,power,toughness
24,NaN,NaN
27,NaN,NaN
129,NaN,NaN
143,NaN,4.0
178,NaN,NaN
...,...,...
24860,NaN,3.0
25008,2.0,NaN
25130,NaN,NaN
25321,NaN,NaN


In [44]:
df['power'] = df['power'].fillna(-1).astype(int)
df['toughness'] = df['toughness'].fillna(-1).astype(int)

In [45]:
df.loc[index,['power', 'toughness']].head()

,power,toughness
24,-1,-1
27,-1,-1
129,-1,-1
143,-1,4
178,-1,-1


In [46]:
X = df.drop(columns=['name', 'usd'])
y = df.usd
numerical_features = ['cmc', 'power', 'toughness', 'days_since_release']
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
embedded_features = ['combined_embedding']
# to remember to preprocess when taking an input we would use ohe here
categorical_features = ['rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special', 'rarity_uncommon', 'booster']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', 'passthrough', categorical_features),
        ('embed', 'passthrough', embedded_features),

    ])
X_transformed = preprocessor.fit_transform(X)

In [47]:
data_transformed = pd.DataFrame(X_transformed, columns=numerical_features + categorical_features + embedded_features)
data_transformed.head()

,cmc,power,toughness,days_since_release,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,booster,combined_embedding
0,-0.742891,-0.784331,-0.814794,0.835309,1.0,0.0,0.0,0.0,0.0,1.0,"[-0.007598841097205877, 0.0013788732467219234,..."
1,0.398713,-0.233859,1.30632,0.934263,1.0,0.0,0.0,0.0,0.0,1.0,"[-0.0017277271253988147, 0.000872959615662694,..."
2,0.969514,1.968027,1.836598,0.970833,0.0,1.0,0.0,0.0,0.0,0.0,"[-0.0017277271253988147, 0.000872959615662694,..."
3,-1.884494,-0.784331,-0.814794,-1.388084,1.0,0.0,0.0,0.0,0.0,1.0,"[-0.010446079075336456, 0.002996041439473629, ..."
4,-0.172089,-0.784331,-0.814794,0.693332,0.0,0.0,1.0,0.0,0.0,0.0,"[-0.007598841097205877, 0.0013788732467219234,..."


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=42)